In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import BaseLogger
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import numpy as np
import json
import os

In [2]:
class TrainingM(BaseLogger):
    def __init__(self, figPath, jsonPath = None, startAt = 0):
        
        super(TrainingM, self).__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt
        
    def on_train_begin(self, logs={}):
        
        self.H = {}
        
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())
                
                if self.startAt > 0:
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.startAt]
                        
    def on_epoch_end(self, epoch, logs = {}):
        
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(float(v))
            self.H[k] = l
            
        if self.jsonPath is not None:
            f = open(self.jsonPath, 'w')
            f.write(json.dumps(self.H))
            f.close()
            
            
        if len(self.H['loss']) > 1:
            N = np.arange(0, len(self.H['loss']))
            plt.style.use('ggplot')
            plt.figure()
            plt.plot(N, self.H['loss'], label = 'train_loss')
            plt.plot(N, self.H['val_loss'], label = 'val_loss')
            plt.plot(N, self.H['acc'], label = 'train_acc')
            plt.plot(N, self.H['val_acc'], label = 'val_acc')
            plt.title('Training Loss and Accuracy [Epoch {}]'.format(len(self.H['loss'])))
            plt.xlabel('Epoch #')
            plt.ylabel('Loss/Accuracy')
            plt.legend()
            plt.savefig('Epoch {}.png'.format(epoch+1))
            plt.close()

In [3]:
#importing, scaling reshaping and vectorizing the data set
((trainX, trainY), (testX, testY)) = cifar10.load_data()

#scaling data
trainX = trainX.astype('float')/255.0
testX = testX.astype('float')/255.0


#vectorizing the labels
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

In [4]:
#initializing the label names
labelNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 
             'frog', 'horse', 'ship', 'truck']

In [5]:
#initializing the model
chanDim= -1
def model():
    
    model = Sequential()
    #first layer of Conv2D
    model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(32, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #second layer of Conv2D
    model.add(Conv2D(64, (3,3), padding = 'same', input_shape = (32, 32, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(Conv2D(64, (3,3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis = chanDim))
    model.add(BatchNormalization(axis = chanDim))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    #FC layet
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model

In [6]:
#initializing the optimizer and model
opt = SGD(lr = 0.01, momentum = 0.9, nesterov = True)
model = model()
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________

In [7]:
figPath = os.path.sep.join(['output', '{}.png'.format(os.getpid())])
jsonPath = os.path.sep.join(['output', '{}.json'.format(os.getpid())])
callbacks = [TrainingM(figPath, jsonPath = jsonPath)]
H = model.fit(trainX, trainY, validation_data = (testX, testY), 
              batch_size = 64, epochs = 100, callbacks = callbacks, verbose = 1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 13s 261us/sample - loss: 1.6255 - acc: 0.4565 - val_loss: 1.2708 - val_acc: 0.5571
Epoch 2/100
50000/50000 [==============================] - 11s 222us/sample - loss: 1.1722 - acc: 0.5967 - val_loss: 1.0322 - val_acc: 0.6435
Epoch 3/100
50000/50000 [==============================] - 11s 224us/sample - loss: 0.9954 - acc: 0.6571 - val_loss: 0.8341 - val_acc: 0.7090
Epoch 4/100
50000/50000 [==============================] - 11s 218us/sample - loss: 0.8853 - acc: 0.6937 - val_loss: 1.0053 - val_acc: 0.6626
Epoch 5/100
50000/50000 [==============================] - 11s 217us/sample - loss: 0.8060 - acc: 0.7196 - val_loss: 0.7552 - val_acc: 0.7429
Epoch 6/100
50000/50000 [==============================] - 11s 215us/sample - loss: 0.7419 - acc: 0.7421 - val_loss: 0.7042 - val_acc: 0.7588
Epoch 7/100
50000/50000 [==============================] - 11s 217us/sample - loss: 0.6952 - acc: 

50000/50000 [==============================] - 11s 221us/sample - loss: 0.1358 - acc: 0.9518 - val_loss: 0.6933 - val_acc: 0.8223
Epoch 59/100
50000/50000 [==============================] - 11s 223us/sample - loss: 0.1285 - acc: 0.9546 - val_loss: 0.6722 - val_acc: 0.8267
Epoch 60/100
50000/50000 [==============================] - 11s 222us/sample - loss: 0.1355 - acc: 0.9527 - val_loss: 0.6461 - val_acc: 0.8290
Epoch 61/100
50000/50000 [==============================] - 11s 224us/sample - loss: 0.1249 - acc: 0.9563 - val_loss: 0.6367 - val_acc: 0.8323
Epoch 62/100
50000/50000 [==============================] - 11s 225us/sample - loss: 0.1300 - acc: 0.9543 - val_loss: 0.6400 - val_acc: 0.8309
Epoch 63/100
50000/50000 [==============================] - 11s 222us/sample - loss: 0.1258 - acc: 0.9559 - val_loss: 0.6680 - val_acc: 0.8296
Epoch 64/100
50000/50000 [==============================] - 11s 222us/sample - loss: 0.1235 - acc: 0.9564 - val_loss: 0.6784 - val_acc: 0.8228
Epoch 65/100